In [ ]:
import pandas as pd
import os

In [ ]:
mrs02_transformer_path = '/Users/jk1/temp/opsum_prediction_output/transformer/3M_mrs02/transformer_20230402_184459_test_set_evaluation/overall_results.csv'
mrs02_xgb_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/3M_mrs02/with_feature_aggregration/testing/test_XGB_results.csv'
mrs02_lstm_path = '/Users/jk1/temp/opsum_prediction_output/LSTM_72h/testing/3M_mRS02/2023_01_02_1057/test_LSTM_sigmoid_all_balanced_0.2_2_True_RMSprop_3M mRS 0-2_16_3/test_LSTM_results.tsv'
mrs02_mlp_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_perceptron/mrs02/with_feature_aggregration_mlp/all_folds/test_mlp_cv_2_results.csv'
mrs02_thrivec_path = '/Users/jk1/temp/opsum_prediction_output/THRIVE_C/THRIVE_C_3m_mrs02_predictions/thriveC_mrs02_results.csv'

death_xgb_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/3M_Death/testing/test_XGB_results.csv'
death_transformer_path = '/Users/jk1/temp/opsum_prediction_output/transformer/3M_Death/testing/overall_results.csv'
death_lstm_path = '/Users/jk1/temp/opsum_prediction_output/LSTM_72h/testing/Death_in_hospital/2024_02_05_1346/LSTM_test_death_in_hospital/test_LSTM_sigmoid_all_unchanged_0.2_1_True_RMSprop_Death in hospital_32_3/test_LSTM_results_fold_3.tsv'
death_mlp_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_perceptron/death/feature_aggregration_mlp/testing/test_mlp_cv_2_results.csv'
death_thrivec_path = '/Users/jk1/temp/opsum_prediction_output/THRIVE_C/THRIVE_C_3m_death_predictions/3m_death_results.csv'

death_mimic_xgb_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/3M_Death/external_validation/external_validation_XGB_results.csv'
death_mimic_transformer_path = '/Users/jk1/temp/opsum_prediction_output/transformer/3M_Death/external_validation/overall_results.csv'
death_mimic_lstm_path = '/Users/jk1/temp/opsum_prediction_output/LSTM_72h/external_validation/3M_Death/2023_01_04_2020/all_folds/test_LSTM_sigmoid_all_balanced_0.0_2_True_RMSprop_3M Death_128_2/test_LSTM_results_fold_2.tsv'
death_mimic_mlp_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_perceptron/death/feature_aggregration_mlp/external_validation/overall_results.csv'

death_in_hospital_xgb_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/Death_in_hospital/testing/test_XGB_cv_2_results.csv'
death_in_hospital_transformer_path = '/Users/jk1/temp/opsum_prediction_output/transformer/Death_in_hospital/testing/15_02_23/overall_results.csv'
death_in_hospital_lstm_path = '/Users/jk1/temp/opsum_prediction_output/LSTM_72h/testing/Death_in_hospital/2024_02_05_1346/LSTM_test_death_in_hospital/test_LSTM_sigmoid_all_unchanged_0.2_1_True_RMSprop_Death in hospital_32_3/test_LSTM_results_fold_3.tsv'
death_in_hospital_thrivec_path = '/Users/jk1/temp/opsum_prediction_output/THRIVE_C/THRIVE_C_death_in_hospital_predictions/death_in_hospital_results.csv'

death_in_hospital_mimic_xgb_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/Death_in_hospital/external_validation/external_validation_cv_2_results.csv'
death_in_hospital_mimic_transformer_path = '/Users/jk1/temp/opsum_prediction_output/transformer/Death_in_hospital/external_validation/overall_results.csv'
death_in_hospital_mimic_lstm_path = '/Users/jk1/temp/opsum_prediction_output/LSTM_72h/external_validation/Death_in_hospital/2024_02_24/test_LSTM_sigmoid_all_unchanged_0.2_1_True_RMSprop_Death in hospital_32_3/test_LSTM_results_fold_3.tsv'
death_in_hospital_mimic_thrivec_path = '/Users/jk1/temp/opsum_prediction_output/THRIVE_C/THRIVE_C_death_in_hospital_mimic_predictions/mimic_Death_in_hospital_baseline_score_results.csv'

In [ ]:
output_dir = '/Users/jk1/Downloads'

In [ ]:
mrs02_xgb_df = pd.read_csv(mrs02_xgb_path)
mrs02_transformer_df = pd.read_csv(mrs02_transformer_path)
mrs02_lstm_df = pd.read_csv(mrs02_lstm_path, sep='\t')
mrs02_mlp_df = pd.read_csv(mrs02_mlp_path)
mrs02_thrivec_df = pd.read_csv(mrs02_thrivec_path)

death_xgb_df = pd.read_csv(death_xgb_path)
death_transformer_df = pd.read_csv(death_transformer_path)
death_lstm_df = pd.read_csv(death_lstm_path, sep='\t')
death_mlp_df = pd.read_csv(death_mlp_path)
death_thrivec_df = pd.read_csv(death_thrivec_path)

death_mimic_xgb_df = pd.read_csv(death_mimic_xgb_path)
death_mimic_transformer_df = pd.read_csv(death_mimic_transformer_path)
death_mimic_lstm_df = pd.read_csv(death_mimic_lstm_path, sep='\t')
death_mimic_mlp_df = pd.read_csv(death_mimic_mlp_path)

death_in_hospital_xgb_df = pd.read_csv(death_in_hospital_xgb_path)
death_in_hospital_transformer_df = pd.read_csv(death_in_hospital_transformer_path)
death_in_hospital_lstm_df = pd.read_csv(death_in_hospital_lstm_path, sep='\t')
death_in_hospital_thrivec_df = pd.read_csv(death_in_hospital_thrivec_path)

death_in_hospital_mimic_xgb_df = pd.read_csv(death_in_hospital_mimic_xgb_path)
death_in_hospital_mimic_transformer_df = pd.read_csv(death_in_hospital_mimic_transformer_path)
death_in_hospital_mimic_lstm_df = pd.read_csv(death_in_hospital_mimic_lstm_path, sep='\t')
death_in_hospital_mimic_thrivec_df = pd.read_csv(death_in_hospital_mimic_thrivec_path)

In [ ]:
mrs02_transformer_df = mrs02_transformer_df[mrs02_transformer_df.fold == mrs02_transformer_df.best_cv_fold.mode()[0]]
death_transformer_df = death_transformer_df[death_transformer_df.fold == death_transformer_df.best_cv_fold.mode()[0]]
death_mimic_transformer_df = death_mimic_transformer_df[death_mimic_transformer_df.fold == death_mimic_transformer_df.best_cv_fold.mode()[0]]
death_in_hospital_transformer_df = death_in_hospital_transformer_df[death_in_hospital_transformer_df.fold == death_in_hospital_transformer_df.best_cv_fold.mode()[0]]
death_in_hospital_mimic_transformer_df = death_in_hospital_mimic_transformer_df[death_in_hospital_mimic_transformer_df.fold == death_in_hospital_mimic_transformer_df.best_cv_fold.mode()[0]]

death_in_hospital_mimic_thrivec_df = death_in_hospital_mimic_thrivec_df[death_in_hospital_mimic_thrivec_df.method_name == 'THRIVEC']
# death_mimic_mlp_df = death_mimic_mlp_df[death_mimic_mlp_df.fold == death_mimic_mlp_df.best_val_fold.mode()[0] - 1]

In [ ]:
def preprocess_df(df, model_name, outcome, dataset_name):
    preprocessed_df = pd.DataFrame()
    # report result rounded to 3 decimal places with 95% CI
    preprocessed_df['ROC AUC'] = [f'{round(df["auc_test"].values[0], 3):.3f} ({round(df["auc_test_lower_ci"].values[0], 3):.3f}-{round(df["auc_test_upper_ci"].values[0], 3):.3f})']
    preprocessed_df["Matthew's Coefficient"] = [f'{round(df["matthews_test"].values[0], 3):.3f} ({round(df["matthews_test_lower_ci"].values[0], 3):.3f}-{round(df["matthews_test_upper_ci"].values[0], 3):.3f})']
    preprocessed_df["Accuracy"] = [f'{round(df["accuracy_test"].values[0], 3):.3f} ({round(df["accuracy_test_lower_ci"].values[0], 3):.3f}-{round(df["accuracy_test_upper_ci"].values[0], 3):.3f})']
    preprocessed_df["Precision (positive predictive value)"] = [f'{round(df["precision_test"].values[0], 3):.3f} ({round(df["precision_test_lower_ci"].values[0], 3):.3f}-{round(df["precision_test_upper_ci"].values[0], 3):.3f})']
    preprocessed_df["Recall (Sensitivity)"] = [f'{round(df["recall_test"].values[0], 3):.3f} ({round(df["recall_test_lower_ci"].values[0], 3):.3f}-{round(df["recall_test_upper_ci"].values[0], 3):.3f})']
    if 'specificity_test' in df.columns:
        preprocessed_df["Specificity"] = [f'{round(df["specificity_test"].values[0], 3):.3f} ({round(df["specificity_test_lower_ci"].values[0], 3):.3f}-{round(df["specificity_test_upper_ci"].values[0], 3):.3f})']
    else:
        preprocessed_df["Specificity"] = [f'NA']

    preprocessed_df['Model'] = [model_name]
    preprocessed_df['Outcome'] = [outcome]
    preprocessed_df['Dataset'] = [dataset_name]

    return preprocessed_df

In [ ]:
processed_mrs02_xgb_df = preprocess_df(mrs02_xgb_df, 'XGB', '3M mrs02', 'GSU')
processed_death_xgb_df = preprocess_df(death_xgb_df, 'XGB', '3M Death', 'GSU')
processed_death_mimic_xgb_df = preprocess_df(death_mimic_xgb_df, 'XGB', '3M Death', 'MIMIC')
preprocessed_death_in_hospital_xgb_df = preprocess_df(death_in_hospital_xgb_df, 'XGB', 'Death in hospital', 'GSU')
preprocessed_death_in_hospital_mimic_xgb_df = preprocess_df(death_in_hospital_mimic_xgb_df, 'XGB', 'Death in hospital', 'MIMIC')

In [ ]:
processed_mrs02_transformer_df = preprocess_df(mrs02_transformer_df, 'Transformer', '3M mrs02', 'GSU')
processed_death_transformer_df = preprocess_df(death_transformer_df, 'Transformer', '3M Death', 'GSU')
processed_death_mimic_transformer_df = preprocess_df(death_mimic_transformer_df, 'Transformer', '3M Death', 'MIMIC')
processed_death_in_hospital_transformer_df = preprocess_df(death_in_hospital_transformer_df, 'Transformer', 'Death in hospital', 'GSU')
processed_death_in_hospital_mimic_transformer_df = preprocess_df(death_in_hospital_mimic_transformer_df, 'Transformer', 'Death in hospital', 'MIMIC')

In [ ]:
processed_mrs02_lstm_df = preprocess_df(mrs02_lstm_df, 'LSTM', '3M mrs02', 'GSU')
processed_death_lstm_df = preprocess_df(death_lstm_df, 'LSTM', '3M Death', 'GSU')
processed_death_mimic_lstm_df = preprocess_df(death_mimic_lstm_df, 'LSTM', '3M Death', 'MIMIC')
processed_death_in_hospital_lstm_df = preprocess_df(death_in_hospital_lstm_df, 'LSTM', 'Death in hospital', 'GSU')
processed_death_in_hospital_mimic_lstm_df = preprocess_df(death_in_hospital_mimic_lstm_df, 'LSTM', 'Death in hospital', 'MIMIC')

In [ ]:
# processed_mrs02_mlp_df = preprocess_df(mrs02_mlp_df, 'MLP', '3M mrs02', 'GSU')
# processed_death_mlp_df = preprocess_df(death_mlp_df, 'MLP', '3M Death', 'GSU')
# processed_death_mimic_mlp_df = preprocess_df(death_mimic_mlp_df, 'MLP', '3M Death', 'MIMIC')

In [ ]:
processed_mrs02_thrivec_df = preprocess_df(mrs02_thrivec_df, 'THRIVE-C', '3M mrs02', 'GSU')
processed_death_thrivec_df = preprocess_df(death_thrivec_df, 'THRIVE-C', '3M Death', 'GSU')
processed_death_in_hospital_thrivec_df = preprocess_df(death_in_hospital_thrivec_df, 'THRIVE-C', 'Death in hospital', 'GSU')
processed_death_in_hospital_mimic_thrivec_df = preprocess_df(death_in_hospital_mimic_thrivec_df, 'THRIVEC', 'Death in hospital', 'MIMIC')

In [ ]:
overall_mrs02_results_df = pd.concat([processed_mrs02_transformer_df, processed_mrs02_xgb_df, processed_mrs02_lstm_df, processed_mrs02_thrivec_df])

In [ ]:
overall_death_results_df = pd.concat([processed_death_transformer_df, processed_death_xgb_df, processed_death_lstm_df, processed_death_thrivec_df])

In [ ]:
overall_death_mimic_results_df = pd.concat([processed_death_mimic_transformer_df, processed_death_mimic_xgb_df, processed_death_mimic_lstm_df])

In [ ]:
overall_death_in_hospital_results_df = pd.concat([processed_death_in_hospital_transformer_df, preprocessed_death_in_hospital_xgb_df, processed_death_in_hospital_lstm_df, processed_death_in_hospital_thrivec_df])
overall_death_in_hospital_mimic_results_df = pd.concat([processed_death_in_hospital_mimic_transformer_df, preprocessed_death_in_hospital_mimic_xgb_df, processed_death_in_hospital_mimic_lstm_df, processed_death_in_hospital_mimic_thrivec_df])

In [ ]:
overall_mrs02_results_df

In [ ]:
overall_death_results_df

In [ ]:
overall_death_mimic_results_df

In [ ]:
overall_death_in_hospital_results_df

In [ ]:
overall_death_in_hospital_mimic_results_df

In [ ]:
# save results as csv
overall_mrs02_results_df.to_csv(os.path.join(output_dir, 'mrs02_model_comparison.csv'), index=False)
overall_death_results_df.to_csv(os.path.join(output_dir, 'death_model_comparison.csv'), index=False)
overall_death_mimic_results_df.to_csv(os.path.join(output_dir, 'death_mimic_model_comparison.csv'), index=False)
overall_death_in_hospital_results_df.to_csv(os.path.join(output_dir, 'death_in_hospital_model_comparison.csv'), index=False)
overall_death_in_hospital_mimic_results_df.to_csv(os.path.join(output_dir, 'death_in_hospital_mimic_model_comparison.csv'), index=False)

In [ ]:
# save joined table
overall_results_df = pd.concat([overall_mrs02_results_df, overall_death_results_df, overall_death_mimic_results_df, overall_death_in_hospital_results_df, overall_death_in_hospital_mimic_results_df])
overall_results_df.to_csv(os.path.join(output_dir, 'overall_model_comparison.csv'), index=False)